In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from tsfresh import extract_features

carotid_df = pd.read_csv('data/carotid_pressure.csv')
illiac_df = pd.read_csv('data/illiac_pressure.csv')

#Rename the first column(s)
carotid_df.rename(columns={'Unnamed: 0':'patient_id'}, inplace=True)
illiac_df.rename(columns={'Unnamed: 0':'patient_id'}, inplace=True)


# X and Y split
#y = carotid_df[['patient_id','target']].copy()
y = carotid_df[['target']].copy()
X_carotid = carotid_df.copy()
X_carotid = X_carotid.drop('target', axis=1)
X_illiac = illiac_df.copy()
X_illiac = X_illiac.drop('target', axis=1)

patient_id = carotid_df[['patient_id']].copy()
X_carotid_series_only = X_carotid.drop(X_carotid.columns[0],axis=1)
X_illiac_series_only = X_illiac.drop(X_illiac.columns[0],axis=1)


#Cleanup
X_carotid_series_only = X_carotid_series_only.interpolate(method="linear")
X_carotid_series_only = X_carotid_series_only.interpolate(method="bfill")
sigma_row = X_carotid_series_only.std()
sigma_avg = np.mean(sigma_row)

rows=0
for index,row in X_carotid_series_only.iterrows():
    X_carotid_series_only.iloc[rows,:] = scipy.ndimage.gaussian_filter1d(X_carotid_series_only.iloc[rows,:], sigma_avg)
    rows += 1

    
X_illiac_series_only = X_illiac_series_only.interpolate(method="linear")
X_illiac_series_only = X_illiac_series_only.interpolate(method="bfill")
sigma_row = X_illiac_series_only.std()
sigma_avg = np.mean(sigma_row)

rows=0
for index,row in X_illiac_series_only.iterrows():
    X_illiac_series_only.iloc[rows,:] = scipy.ndimage.gaussian_filter1d(X_illiac_series_only.iloc[rows,:], sigma_avg)
    rows += 1


#Re-join the dataframe  
#X_carotid = pd.concat([patient_id, X_carotid_series_only], axis=1)
X_carotid = X_carotid_series_only
#X_illiac = pd.concat([patient_id, X_illiac_series_only], axis=1)
X_illiac = X_illiac_series_only
  
    
# Train and test sets splitting
X_train_carotid, X_test_carotid, y_train, y_test = train_test_split(X_carotid, y, test_size = 0.20, random_state = 0)
X_train_illiac, X_test_illiac, y_train, y_test = train_test_split(X_illiac, y, test_size = 0.20, random_state = 0)

#y_train = np.ravel(y_train)
#y_test = np.ravel(y_test)


# Extract statistical features for Train sets
#extracted_features = extract_features(timeseries, column_id="id", column_sort="time")
    
    
    
#Scale the data    
#scaler = StandardScaler()
#X_train_carotid = scaler.fit_transform(X_train_carotid)
#X_train_illiac = scaler.fit_transform(X_train_illiac)
#X_test_carotid = scaler.fit_transform(X_test_carotid)
#X_test_illiac = scaler.fit_transform(X_test_illiac)


# Combine Carotid and Illiac inputs. This might also be tricky...be creative here!
# X_train = np.hstack((X_train_carotid, X_train_illiac))
X_train = np.hstack((X_train_carotid, X_train_illiac))
#X_train = X_train_carotid

X_test = np.hstack((X_test_carotid, X_test_illiac))
#X_test = np.hstack((X_test_carotid, X_test_illiac))
#X_test = X_test_carotid
        
# # Training the Naive Bayes model on the Training set
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)


# # Predicting the Test set results
y_pred = classifier.predict(X_test)

        
# # Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
        
stats =  accuracy_score(y_test, y_pred)
modelLearn = True


print(stats)

#print(X_carotid)
#print(len(X_test_illiac))
#print(X_test_illiac)
#print(np.shape(X))
#print(X)

#plt.plot(X_carotid[100])
#plt.show()

#print(row)
#plt.plot(X_train_carotid[0])
#X_carotid.iloc[100:101,:].T.plot()
#plt.show()


/tmp/ipykernel_8113/951108043.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


0.51931330472103
